In [ ]:
import numpy as np
import pandas as pd
from datasets import load_dataset, concatenate_datasets, Dataset, DatasetDict


Import Datasets

In [42]:
ds1 = load_dataset("ccdv/govreport-summarization")
ds2 = load_dataset("FiscalNote/billsum")

# GovReport splits (report, summary)
gov_train = ds1["train"].rename_columns({"report": "text"})
gov_val   = ds1["validation"].rename_columns({"report": "text"})
gov_test  = ds1["test"].rename_columns({"report": "text"})

# BillSum splits (text, summary, title)
bill_train = ds2["train"]
bill_test  = ds2["test"]
bill_ca_test = ds2["ca_test"]  # treat as out of domain test for model generalisation
split = bill_test.train_test_split(test_size=0.5, seed=42)
bill_val = split["train"]
bill_test = split["test"]

# Combined (text, summary, source)
comb_train = concatenate_datasets([gov_train, bill_train.select_columns(["text", "summary"])])
comb_val = concatenate_datasets([gov_val, bill_val.select_columns(["text", "summary"])])
comb_test = concatenate_datasets([gov_test, bill_test.select_columns(["text", "summary"])])
# add source column
comb_train = comb_train.add_column("source",["govreport"] * len(gov_train) + ["billsum"] * len(bill_train))
comb_val = comb_val.add_column("source",["govreport"] * len(gov_val) + ["billsum"] * len(bill_val))
comb_test = comb_test.add_column("source",["govreport"] * len(gov_test) + ["billsum"] * len(bill_test))
# shuffle to avoid bias
comb_train = comb_train.shuffle(seed=42)
comb_val = comb_val.shuffle(seed=42)
comb_test = comb_test.shuffle(seed=42)


Data Cleaning and Preprocessing

Feature Engineering

Exploratory Data Analysis

Modeling